# Selected optimization topics

  * Avoid unneeded data copies
    * emplace_back vs push_back
    * Pass and receive by reference      
  * Improve cache locality
    * Avoid complicated data structures
    * SoA vs AoS
  * Do less work at run time
    * Redundant computations
    * Compile time 
    

In [1]:
//Lets make a simple class that has a printout when it is created

class Widget {
public:
    Widget(int in_w) { w=in_w; std::cout << "New widget1\n"; }
    Widget(const Widget&) { std::cout << "New widget2\n"; }
    ~Widget() { }
private:
    int w; 
};


In [2]:
Widget a(10);

New widget1


Now a vector of them

In [9]:
std::vector<Widget> my_widgets(5,(10));
std::cout << "And reserve more space..\n";
my_widgets.reserve(7);

New widget1
New widget2
New widget2
New widget2
New widget2
New widget2
And reserve more space..
New widget2
New widget2
New widget2
New widget2
New widget2


I can then extend the vector directly

In [10]:
my_widgets.push_back(Widget(10));

New widget1
New widget2


A temporary Widget got created, so we did 2x the work. In this case ```emplace_back``` can help

In [8]:
my_widgets.emplace_back(10);

New widget1
